In [22]:
import pandas as pd
df=pd.read_csv('../data/raw/online_retail.csv')


### Remove cancelled invoices

In [23]:
df=df[~df['InvoiceNo'].str.startswith('C')]

### Remove invalid Quantity & UnitPrice

In [24]:
df=df[(df['Quantity']>0) & (df['UnitPrice']>0)]

### Remove missing CustomerID

In [25]:
df=df.dropna(subset=['CustomerID'])

### Convert InvoiceDate to datetime

In [26]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

### Focus only on UK customers

In [27]:
df=df[df['Country']=='United Kingdom']

### Remove extreme Quantity Outliers [IQR]

In [28]:
Q1=df['Quantity'].quantile(0.25)
Q3=df['Quantity'].quantile(0.75)
IQR=Q3-Q1
upper_bound=Q3+1.5*IQR

df=df[df['Quantity']<=upper_bound]

In [29]:
df.to_csv('../data/processed/online_retail_cleaned.csv',index=False)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 334073 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    334073 non-null  object        
 1   StockCode    334073 non-null  object        
 2   Description  334073 non-null  object        
 3   Quantity     334073 non-null  int64         
 4   InvoiceDate  334073 non-null  datetime64[ns]
 5   UnitPrice    334073 non-null  float64       
 6   CustomerID   334073 non-null  float64       
 7   Country      334073 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 22.9+ MB


## RFM Feature Engineering

In [31]:
df=pd.read_csv("../data/processed/online_retail_cleaned.csv")

In [33]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

### Calculate the refrence date [one day after last transaction to be used in Recency]

In [34]:
refrence_date=df['InvoiceDate'].max()+pd.Timedelta(days=1)

### Create Monetary Value per transaction

In [35]:
df['TotalPrice']=df['Quantity']*df['UnitPrice']

### Aggregate to customer level

In [38]:
rfm=df.groupby("CustomerID").agg({
  "InvoiceDate": lambda x: (refrence_date-x.max()).days,
  "InvoiceNo": "nunique",
  "TotalPrice": "sum"
}).reset_index()

### Rename columns

In [39]:
rfm.columns=["CustomerID","Recency","Frequency","Monetary"]

In [40]:
rfm.describe()

,CustomerID,Recency,Frequency,Monetary
count,3825.000000,3825.000000,3825.000000,3825.000000
mean,15560.726275,92.133856,4.051503,1169.564477
std,1572.037298,99.286178,6.662454,2240.842468
min,12747.000000,1.000000,1.000000,1.900000
25%,14210.000000,18.000000,1.000000,243.220000
50%,15569.000000,51.000000,2.000000,551.950000
75%,16907.000000,144.000000,4.000000,1293.350000
max,18287.000000,374.000000,203.000000,61295.620000


In [41]:
rfm.to_csv("../data/processed/rfm_features.csv", index=False)

## RFM Feature Engineering

After cleaning the transactional data, customer-level features were constructed
using the RFM framework:

- **Recency**: Number of days since the customer’s most recent purchase
- **Frequency**: Number of unique invoices per customer
- **Monetary**: Total amount spent by the customer

RFM features provide a compact and interpretable representation of customer
purchasing behavior and are widely used for customer segmentation tasks.

The resulting dataset aggregates transactions at the customer level and
serves as the input for clustering algorithms.
